In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
import csv, pydot, graphviz
print('Pandas version: ', pd.__version__)
print('Numpy version: ', np.__version__)

In [ ]:
data=pd.read_csv('../dataset/TPE_model_features.csv')
data = data.loc[data['鄉鎮市區'] == '中山區']
data.insert(80, 'y', data['單價元平方公尺'])
data.drop(['單價元平方公尺'],axis=1,inplace=True)
data.head()

In [ ]:
print(data.info())

In [ ]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)
data.drop(['Unnamed: 0.1'],axis=1,inplace=True)
data.drop(['鄉鎮市區'],axis=1,inplace=True)
data.drop(['geometry'],axis=1,inplace=True)
#data = data[['交易年份','屋齡','交易標的','建物現況格局-房','建物現況格局-廳','建物現況格局-衛','車位類別','電梯','floor','NEAR_HOSTIPAL_500','CLINIC_COUNT_500','DENTIST_COUNT_500','PHARMACY_COUNT_500','CSTORE_COUNT_500','FASTFOOD_COUNT_500','LIBRARY_COUNT_500','NEAR_UNIVERSITY_500','NEAR_FIRESTATION_500','NEAR_FUEL_500','NEAR_MARKET_500','NEAR_POLICE_500','TEMPLE_COUNT_500','CEMETERY_AREA_500','WATER_AREA_500','PARKING_AREA_500','STOP_COUNT_500','NEAR_MRT_500','y']]
data.dtypes

In [ ]:
# 2.刪除標準差為0的欄位
#data.drop(['NEAR_SCHOOL_500'],axis=1,inplace=True)
#data.drop(['NEAR_SCHOOL_250'],axis=1,inplace=True)
data.drop(['NEAR_SCHOOL_750'],axis=1,inplace=True)
#data.drop(['NEAR_MARKET_250'],axis=1,inplace=True)
#data.drop(['NEAR_MARKET_500'],axis=1,inplace=True)
#data.drop(['NEAR_MARKET_750'],axis=1,inplace=True)
data.drop(['NEAR_LRT_250'],axis=1,inplace=True)
data.drop(['NEAR_LRT_500'],axis=1,inplace=True)
data.drop(['NEAR_LRT_750'],axis=1,inplace=True)
data.drop(['NEAR_TRA_250'],axis=1,inplace=True)
#data.drop(['NEAR_TRA_500'],axis=1,inplace=True)
#data.drop(['NEAR_TRA_750'],axis=1,inplace=True)
#data.drop(['CEMETERY_AREA_250'],axis=1,inplace=True)
#data.drop(['CEMETERY_AREA_500'],axis=1,inplace=True)
#data.drop(['CEMETERY_AREA_750'],axis=1,inplace=True)
#data.drop(['NEAR_POLICE_250'],axis=1,inplace=True)
#data.drop(['NEAR_POLICE_500'],axis=1,inplace=True)
#data.drop(['NEAR_POLICE_750'],axis=1,inplace=True)

In [ ]:
data = data.sample(frac=1.0)
data = data.reset_index()
train_data = data.sample(frac=0.8)
train_data

In [ ]:
data2 = data[~data.index.isin(train_data.index)]

In [ ]:
val_data = data2.sample(frac=0.5)
test_data = data2[~data2.index.isin(val_data.index)]

In [ ]:
train_data.drop(['index'],axis=1,inplace=True)
val_data.drop(['index'],axis=1,inplace=True)
test_data.drop(['index'],axis=1,inplace=True)

In [ ]:
# 1.做到這邊要檢視資料!!!!!!!!!!!回到2.刪除標差為0.
train_validation_data = pd.concat([train_data, val_data])
mean = train_validation_data.mean()
std = train_validation_data.std()
train_data = (train_data-mean)/std
val_data = (val_data -mean)/std
#print(mean)
mean_df = pd.DataFrame(mean)
mean_df.to_csv('mean.csv')
std_df = pd.DataFrame(std)
std_df.to_csv('std.csv')

In [ ]:
#修改X_data = np.array(train_data[train_data.columns[0:?]])
#X_val = np.array(val_data[val_data.columns[0:?]])
X_data = np.array(train_data[train_data.columns[0:72]])
y_data = np.array(train_data[train_data.columns[-1]]).reshape(len(train_data),1)
X_val = np.array(val_data[val_data.columns[0:72]])
y_val = np.array(val_data[val_data.columns[-1]]).reshape(len(val_data),1)

print(X_data.shape,type(X_data))
print(y_data.shape,type(y_data))

In [ ]:
#檢視std.CSV修改model.add(keras.layers.Dense(?, activation='relu', input_shape=(?,)))
model = tf.keras.Sequential()
model.add(keras.layers.Dense(71, activation='relu', input_shape=(71,))) 
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dense(1)) 

In [ ]:
model.compile(keras.optimizers.Adam(0.0001),
loss=keras.losses.MeanSquaredError(),
metrics=[keras.metrics.MeanAbsoluteError()])

In [ ]:
model_mckp=keras.callbacks.ModelCheckpoint('TPE_chunshan_model-1.h5',
monitor='val_mean_absolute_error',
save_best_only=True,
mode='min')

In [ ]:
model_cbk=keras.callbacks.TensorBoard()
history = model.fit(X_data, y_data,
batch_size=20,  
epochs=200, 
validation_data=(X_val, y_val),  
callbacks=[model_cbk, model_mckp])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('MSE')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='best')

In [ ]:
plt.plot(history.history['mean_absolute_error'], label='train')
plt.plot(history.history['val_mean_absolute_error'], label='validation')
plt.title('MAE')
plt.ylabel('metrics')
plt.xlabel('epochs')
plt.legend(loc='best')

In [ ]:
#修改x_test = np.array(test_data[test_data.columns[0:?]])
model = keras.models.load_model('TPE_chunshan_model-1.h5')
y_test = np.array(test_data[test_data.columns[-1]]).reshape(len(test_data),1)
test_data = (test_data - mean) / std
x_test = np.array(test_data[test_data.columns[0:71]])
y_pred = model.predict(x_test)
y_pred = np.reshape(y_pred * std['y'] + mean['y'], y_test.shape)
percentage_error = np.mean(np.abs(y_test - y_pred)) / np.mean(y_test) * 100
print("Model Percentage Error: {:.2f}%".format(percentage_error))

In [ ]:
print(type(y_test))
print(type(y_pred))

In [ ]:
y_test = pd.DataFrame(y_test)
y_pred = pd.DataFrame(y_pred)
result = pd.concat([y_test, y_pred],axis=1)
result.to_csv('chunshan_result.csv')

In [ ]:
model.layers[0].get_config()

In [ ]:
model.layers[0].get_weights()

In [ ]:
print(model.summary())